# chapter 3. Evaluation

**머신러닝**은 **데이터 가공 / 변환, 모델 학습 / 예측, 그리고 평가의 프로세스로 구성**된다. <br>
머신러닝 모델은 여러 가지 방법으로 예측 성능을 평가할 수 있다. <br>
**성능 평가 지표(Evaluation Metric)는 일반적으로 모델이 분류나 회귀냐에 따라 여러 종류로 나뉜다.** <br>
**회귀**의 경우 대부분실제 값과 예측값의 오차 평균에 기반한다. <br>
**분류**의 평가방법도 일반적으로는 실제 결과 데이터와 예측 결과 데이터가 얼마나 정확하고 오류가 적게 발생하는가에 기반하지만, 정확도만 가지고 판단하는 것은 위험하다. <br>
분류에 사용되는 성능 평가 지표 중 하나인 **이진 분류의 성능 평가 지표**는 0이냐 1이냐 혹은 긍정 / 부정을 판단하고, 정확도보다는 다른 성능 평가 지표가 더 중요시되는 경우가 많다. <br>

**분류의 성능 평가 지표** <br>
1. **정확도** / Accuracy <br>
2. **오차 행렬** / Confusion Matrix <br>
3. **정밀도** / Precision <br> 
4. **재현율** <br> 
5. **F1 스코어** <br> 
6. **ROC AUC** <br> 

분류는 **결정 클래스 값 종류의 유형에 따라** 긍정 / 부정과 같은 2개의 결과값만을 가지는 **이진 분류**와 여러 개의 결정 클래스 값을 가지는 **멀티 분류**로 나뉜다.

## Accuracy 

**정확도**는 **실제 데이터에서 예측 데이터가 얼마나 같은지**를 판단하는 지표이다. <br>
**직관적**으로 **모델 예측 성능을 나타내는 평가 지표**이다. <br>
**이진 분류**의 경우 **데이터의 구성에 따라 ML 모델의 성능을 왜곡할 수 있기 때문에** 정확도 수치 하나만 가지고 성능을 평가하지 않는다. <br>

<img src = 'image/Accuracy.jpg' alt = 'Accruracy' width='500' height='500'>

In [17]:
# 사이킷런의 BaseEstimator를 상속 받으면 Customized 형태의 Estimator를 개발자가 생성할 수 있다. 
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않는다. 
    def fit(self, X, Y = None):
        pass
    
    # predict() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측한다. 
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        
        return pred

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터 / 테스트 데이터 분할
titanic_df = pd.read_csv(r'/Users/1001l1000/Documents/AI-Tech/Jen/data/titanic_train.csv')
Y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis = 1)
# X_titanic_df = transform_features(X_titanic_df) # Error !!
X_train, X_test, Y_train, Y_test = train_test_split(X_titanic_df, Y_titanic_df,
                                                    test_size = 0.2, random_state = 0)

# 위에 생성한 Dummy Classifier를 이용해 학습 / 예측 / 평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, Y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {0:4f}'.format(accuracy_score(Y_test, mypredictions)))

Dummy Classifier의 정확도는 : 0.385475


정확도를 평가 지표로 사용할 때는 매우 신중해야 한다. <br>
불균형한(Imbalanced) 레이블 값 분포에서 ML 모델의 성능을 판단할 경우, 적합한 평가 지표가 아니다. <br>
 
사이킷런은 **load_digits() API를 통해 MNIST 데이터 세트를 제공**한다. 

In [26]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, Y):
        pass
    
    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0 값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype = bool)
    
# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()
    
# digits 번호가 7번이면 True이고, 이를 astype(int)로 1 변환, 7번이 아니면 False이고 0으로 변환
Y = (digits.target == 7).astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(digits.data, Y, random_state = 11)

In [29]:
# 불균형한 레이블 데이터 분포도 확인 
print('레이블 테스트 세트 크기 : ', Y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(Y_test).value_counts())

# Dummy Classifier로 학습 / 예측 / 정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, Y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 : {:.3f}'.format(accuracy_score(Y_test, fakepred)))

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는 : 0.900


정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용돼서는 안된다. <br>
여러가지 분류 지표와 함께 적용하여 ML 모델 성능을 평가해야 한다. 